In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/anantwag19/datascience_project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="anantwag19"
os.environ["MLFLOW_TRACKING_PASSWORD"]="1f02ee57d9a75d0f6eb7dda3aa725319dbe06609"

  Using cached dagshub-0.5.9-py3-none-any.whl.metadata (12 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl.metadata (9.0 kB)
  Using cached rich-14.0.0-py3-none-any.whl.metadata (18 kB)
  Using cached dacite-1.6.0-py3-none-any.whl.metadata (14 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached gql-3.5.2-py2.py3-none-any.whl.metadata (9.4 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached treelib-1.7.1-py3-none-any.whl.metadata (1.4 kB)
  Using cached pathvalidate-3.2.3-py3-none-any.whl.metadata (12 kB)
  Using cached semver-3.0.4-py3-none-any.whl.metadata (6.8 kB)
  Using cached dagshub_annotation_converter-0.1.9-py3-none-any.whl.metadata (2.5 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached marshmallow-3.26.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached typing_inspect-0.9.0-py3-none-any.whl.metadat

NameError: name 'dagshub' is not defined

In [2]:
import os
%pwd

'e:\\UDemy Final\\MLOPS\\datascienceproject\\research'

In [3]:
os.chdir("../")
%pwd

'e:\\UDemy Final\\MLOPS\\datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str


In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config=ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/krishnaik06/datascienceproject.mlflow"


        )
        return model_evaluation_config


In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [8]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")
    


In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2024-10-09 10:27:03,773: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-10-09 10:27:03,776: INFO: common: yaml file: params.yaml loaded successfully]
[2024-10-09 10:27:03,786: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-10-09 10:27:03,789: INFO: common: created directory at: artifacts]
[2024-10-09 10:27:03,790: INFO: common: created directory at: artifacts/model_evaluation]
[2024-10-09 10:27:05,238: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2024/10/09 10:27:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'ElasticnetModel'.
2024/10/09 10:27:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
2024/10/09 10:27:32 INFO mlflow.tracking._tracking_service.client: 🏃 View run gaudy-sponge-144 at: https://dagshub.com/krishnaik06/datascienceproject.mlflow/#/experiments/0/runs/e6ba693e04de4ad492824b5dd26af2d2.
2024/10/09 10:27:32 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/krishnaik06/datascienceproject.mlflow/#/experiments/0.
